<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2016 Ivania Donoso - Antonio Ossa.</font>
<br>
<font size='1'> Modificado 2025-2 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos

1. [Cómo recorrer o buscar en un grafo](#Cómo-recorrer-o-buscar-en-un-grafo)
    1. [*BFS: Breadth-first search*](#BFS:-Breadth-first-search)
    2. [*DFS: Depth-first search*](#DFS:-Depth-first-search)
        1. [DFS iterativo](#DFS-iterativo)
        2. [DFS recursivo](#DFS-recursivo)


# Cómo recorrer o buscar en un grafo

Podemos recorrer un grafo usando los mismos métodos que utilizamos para árboles (BFS y DFS). Pero debemos tener cuidado en la implementación.

Vamos a realizar estos recorridos sobre el grafo de amistades que ya hemos utilizado.

In [ ]:
# Utilizaremos estas definiciones para Persona, Nodo y Grafo, que ya habíamos revisado anteriormente.
class Persona:

    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    def __repr__(self):
        return self.nombre


class Nodo:

    def __init__(self, valor):
        self.valor = valor

    def __repr__(self):
        return repr(self.valor)


class Grafo:

    def __init__(self, lista_adyacencia=None):
        self.lista_adyacencia = lista_adyacencia or dict()

    def adyacentes(self, x, y):
        return y in self.lista_adyacencia[x]

    def vecinos(self, x):
        return self.lista_adyacencia[x]

    def agregar_vertice(self, x):
        self.lista_adyacencia[x] = set()

    def remover_vertice(self, x):
        self.lista_adyacencia.pop(x, None)
        for k, v in self.lista_adyacencia.items():
            if x in v:
                v.remove(x)

    def agregar_arista(self, x, y):
        if x in self.lista_adyacencia:
            self.lista_adyacencia[x].add(y)

    def remover_arista(self, x, y):
        vecinos_x = self.lista_adyacencia.get(x, set())
        if y in vecinos_x:
            vecinos_x.remove(y)

    def __repr__(self):
        texto_nodos = []
        for nodo, vecinos in self.lista_adyacencia.items():
            texto_nodos.append(f"Amigos de {nodo}: {vecinos}.")
        return "\n".join(texto_nodos)


# Creamos varios nodos que representan a las personas y 
# construimos un grafo para modelar sus relaciones de amistad.
coco = Nodo(Persona("Coco", 15))
thor = Nodo(Persona("Thor", 20))
luna = Nodo(Persona("Luna", 21))
kira = Nodo(Persona("Kira", 20))
bon = Nodo(Persona("Bon", 20))
tomas = Nodo(Persona("Tomás", 10))
anya = Nodo(Persona("Anya", 22))

# Definimos las amistades.
amistades = {
    coco: set([thor, luna, kira, bon, tomas, anya]),
    thor: set([kira, tomas, anya]),
    luna: set([thor, bon, anya]),
    bon: set([luna, tomas, anya]),
    kira: set([thor, luna, bon, anya]),
    anya: set([thor, luna, kira, bon]),
    tomas: set([bon, coco])
}

grafo = Grafo(amistades)
grafo

Amigos de Coco: {Kira, Luna, Bon, Tomás, Anya, Thor}.
Amigos de Thor: {Kira, Anya, Tomás}.
Amigos de Luna: {Thor, Anya, Bon}.
Amigos de Bon: {Tomás, Anya, Luna}.
Amigos de Kira: {Thor, Anya, Bon, Luna}.
Amigos de Anya: {Thor, Bon, Kira, Luna}.
Amigos de Tomás: {Bon, Coco}.

Ahora intentaremos aplicar el mismo algoritmo de BFS que usamos para un árbol. Agregaremos un argumento `limite` para limitar la cantidad de nodos que visitamos:

In [18]:
from collections import deque


def bfs(grafo, inicio, limite=20):
    # La cola de siempre, comienza desde el nodo inicio.
    cola = deque([inicio])

    # Mientras queden vertices por visitar y no nos pasemos del limite de navegación
    while len(cola) > 0 and limite > 0:
        # Obtenemos de la cola el próximo vertice
        vertice = cola.popleft()
        print(vertice)
        # Agregamos los vecinos al stack
        for vecino in grafo[vertice]:
            cola.append(vecino)
        # Visitamos un nodo, bajamos el límite en 1
        limite -= 1

Ahora intentemos recorrer el grafo de amistades anterior, partiendo por Bon:

In [19]:
bfs(amistades, bon)

Bon
Tomás
Anya
Luna
Bon
Coco
Thor
Bon
Kira
Luna
Thor
Anya
Bon
Tomás
Anya
Luna
Kira
Luna
Bon
Tomás


Notamos rápidamente que la salida del programa repite varios nombres. Incluso si aumentamos el valor de `limite`, el problema persiste. ¿Por qué ocurre esto?

Al observar las amistades, descubrimos que existen **ciclos** dentro del grafo: secuencias de personas conectadas que eventualmente regresan al punto de partida. Por ejemplo, Bon y Luna son amigos entre sí, y además Tomás es amigo de Coco, quien es amiga de Luna, que a su vez es amiga de Bon, quien finalmente vuelve a tener de amigo a Tomás. Al recorrer estos vértices, el algoritmo termina visitando nuevamente nodos que ya había procesado, quedando atrapado en un ciclo.

Esto no sucede en los **árboles**, porque su estructura garantiza que existe **una única ruta** para llegar a cada vértice, siempre a través de su nodo padre. Por lo tanto, en un árbol nunca se visita dos veces el mismo vértice con el algoritmo anterior.

En cambio, en un **grafo general** sí pueden existir ciclos, por lo que es fundamental **registrar qué vértices ya han sido visitados**. De lo contrario, el recorrido podría volverse infinito y el programa nunca terminaría. En el ejemplo anterior, si no hubiésemos establecido un límite, el proceso habría continuado indefinidamente.

A continuación, veremos las implementaciones de los algoritmos **BFS** y **DFS**, que incluyen este control para evitar recorrer dos veces el mismo vértice.

## *BFS: Breadth-first search*

BFS recorre exhaustivamente el grafo, dado un punto de partida. Por lo tanto, si un nodo no fue visitado en el recorrido, significa que **no es alcanzable** desde ese punto de partida. 

BFS utiliza una **cola** para mantener el registro de los nodos que debe visitar. Inicialmente el único nodo en la cola es el nodo inicial. Una vez que se extrae un nodo de la cola y se visita, se agregan todos sus vecinos a la cola. Esto provoca que, luego de visitar el nodo inicial, se visitan todos los nodos que están a una arista del inicial; luego se visitan todos los que están a dos aristas del inicial; luego los que están a tres aristas del inicial, y así hasta haber visitado todos los nodos.

Este modo de recorrer hace que cada vez visitamos todos los nodos que están a cierta "distancia" del nodo inicial antes de pasar a la distancia siguiente. Por este recorrido se llama **recorrido en amplitud (_breadth_)**

BFS es práctico cuando se quiere encontrar **la cantidad mínima de aristas a recorrer** para llegar desde un nodo a otro. Esto puede considerarse equivalente a encontrar el "camino más corto" entre dos nodos cuando todos las aristas poseen en el mismo peso. En el caso más general, cuando cada arista puede tener un peso distinto, el criterio de búsqueda de BFS ya no es suficiente y hay otros algoritmos que permiten obtener el camino más corto.


In [20]:
from collections import deque


def bfs(grafo, inicio):
    # Vamos a mantener una lista con los nodos visitados.
    visitados = []
    # La cola de siempre, comienza desde el nodo inicio.
    queue = deque([inicio])

    while len(queue) > 0:
        # Elegimos el siguiente nodo a visitar de la cola
        vertice = queue.popleft()
        # Detalle clave: si ya visitamos el nodo, no hacemos nada!
        if vertice in visitados:
            continue

        # Lo visitamos
        print(vertice)
        visitados.append(vertice)
        # Agregamos los vecinos a la cola si es que no han sido visitados.
        for vecino in grafo[vertice]:
            if vecino not in visitados:
                queue.append(vecino)
    return visitados

In [21]:
bfs(amistades, bon)

Bon
Tomás
Anya
Luna
Coco
Thor
Kira


[Bon, Tomás, Anya, Luna, Coco, Thor, Kira]

Notamos que (1) el algoritmo termina gracias a que mantenemos un registro en `visitados` de todos los nodos revisados y (2) que obtenemos todas las personas como resultado, ya que todas son alcanzables desde alguna otra persona (grafo conexo). 

Pero si cambiamos un poco el grafo, dónde no todos los nodos son alcanzables desde otros nodos, solo obtenemos algunos:

In [22]:
grafo = {
    'A': ['B', 'C'],
    'B': ['D', 'A'],
    'C': ['D'],
    'D': ['B'],
    'E': ['F'],
    'F': ['E'],
    'G': ['A', 'F'],
}

In [23]:
print(f"Nodos alcanzables desde A: {bfs(grafo, 'A')}")

A
B
C
D
Nodos alcanzables desde A: ['A', 'B', 'C', 'D']


In [24]:
print(f"Nodos alcanzables desde E: {bfs(grafo, 'E')}")

E
F
Nodos alcanzables desde E: ['E', 'F']


In [25]:
print(f"Nodos alcanzables desde G: {bfs(grafo, 'G')}")

G
A
F
B
C
E
D
Nodos alcanzables desde G: ['G', 'A', 'F', 'B', 'C', 'E', 'D']


Haz el ejercicio de dibujar el grafo anterior, y ve cómo el algoritmo va recorriendo qué nodos. Como nuestra implementación usa una lista para guardar a los nodos visitados, se mantiene el orden en que fueron agregados. Revisa por qué tiene ese orden los nodos alcanzables de cada resultado.

## *DFS: Depth-first search*

Al igual que BFS, el algoritmo DFS recorre exhaustivamente el grafo, dado un punto de partida. Si un nodo no fue visitado en el recorrido, significa que **no es alcanzable** desde ese punto de partida.

La diferencia con BFS es el orden en que se recorren los nodos. BFS realiza una **búsqueda por amplitud** (o "por nivel"), de manera que primero recorrerá todos los nodos que están a una arista del inicial, después todos los que están a dos aristas del inicial, luego a tres aristas, y así hasta recorrerlos todos.

DFS explora **en profundidad (_depth_)** cada uno de los vecinos de un nodo. Esto significa que a partir del nodo inicial elige uno de sus vecinos y trata de llegar lo más lejos posible del nodo inicial ("desciende por una rama"); una vez que ha llegado lo más lejos posible, se devuelve e intenta otro camino. De esta manera el algoritmo explora cada "rama" completa del grafo antes de probar otra rama.

DFS puede implementarse de manera muy similar a BFS pero cambiando la estructura en que se almacenan los nodos a visitar usando un **_stack_** en lugar de una cola.

DFS también posee una implementación recursiva muy natural. A continuación presentamos ambas implementaciones.

#### DFS iterativo

La implementación iterativa de DFS es muy similar a la de BFS, pero DFS utiliza un _stack_ para mantener el registro de los nodos por visitar, mientras que BFS utiliza una cola. Esta diferencia provoca un cambio en el comportamiento del recorrido.

In [26]:
def dfs(grafo, inicio):
    # Vamos a mantener un set con los nodos visitados.
    visitados = set()

    # El stack de siempre, comienza desde el nodo inicio.
    stack = [inicio]

    while len(stack) > 0:
        vertice = stack.pop()
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vertice in visitados:
            continue

        # Lo visitamos
        print(vertice)
        visitados.add(vertice)

        # Agregamos los vecinos al stack si es que no han sido visitados.
        for vecino in grafo[vertice]:
            if vecino not in visitados:
                stack.append(vecino)

    return list(visitados)

In [27]:
dfs(amistades, bon)

Bon
Luna
Anya
Kira
Thor
Tomás
Coco


[Kira, Luna, Bon, Anya, Tomás, Coco, Thor]

In [28]:
print(f"Nodos alcanzables desde A: {dfs(grafo, 'A')}")

A
C
D
B
Nodos alcanzables desde A: ['A', 'B', 'D', 'C']


In [29]:
print(f"Nodos alcanzables desde E: {dfs(grafo, 'E')}")

E
F
Nodos alcanzables desde E: ['F', 'E']


In [30]:
print(f"Nodos alcanzables desde G: {dfs(grafo, 'G')}")

G
F
E
A
C
D
B
Nodos alcanzables desde G: ['F', 'B', 'A', 'D', 'E', 'C', 'G']


Nota que los resultados (como conjuntos de nodos) son iguales que en BFS, pero el orden es lo que cambia. Revisa en este caso el por qué aparecen en el orden en que imprimen.

#### DFS recursivo

La versión recursiva recorre cada vertice, y se vuelve a llamar recursivamente con cada uno de los vecinos de ese vértice. También se hace necesario maetener en cada llamado una lista con los vértices ya visitados para evitar procesar más de una vez cada vértice.

In [31]:
# Vamos a mantener como parámetro un set con los nodos visitados.
def dfs_recursivo(grafo, vertice, visitados=None):
    visitados = visitados or set()

    # Lo visitamos
    print(vertice)
    visitados.add(vertice)

    for vecino in grafo[vertice]:
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vecino not in visitados:
            dfs_recursivo(grafo, vecino, visitados)

    return list(visitados)

In [32]:
dfs_recursivo(amistades, bon)

Bon
Tomás
Coco
Kira
Thor
Anya
Luna


[Kira, Luna, Bon, Tomás, Anya, Coco, Thor]